# 03b MCU Latency


In [1]:
%run '00_Imports-and-settings.ipynb'

Numpy Version:		 1.19.0
Pandas Version:		 1.0.5
Matplotlib Version:	 3.2.2


In [2]:
# helper function to get model name
def get_model_name(model_string):
    if 'ResNet' in model_string:
        model_name = 'ResNet20_CIFAR-10'
    elif 'LeNet' in model_string:
        model_name = 'LeNet-MNIST'
    else:
        model_name = 'unknown'
        print('unknown model name')
    return model_name

### Load the data

In [3]:
model = 'LeNet-MNIST'
l4_df = pd.read_pickle('results/L4/NUCLEO_L496ZG_LeNet-MNIST_benchmarking+verification_results_2020-07-24.pkl').sort_values(by=['model_type'])
f4_df = pd.read_pickle('results/F4/DISCO_F469NI_LeNet-MNIST_benchmarking+verification_results_2020-07-18.pkl').sort_values(by=['model_type'])
f7_df = pd.read_pickle('results/F7/NUCLEO_F767ZI_LeNet-MNIST_benchmarking+verification_results_2020-07-22.pkl').sort_values(by=['model_type'])

lenet_df = l4_df.append(f4_df)
lenet_df = lenet_df.append(f7_df)

model = '01d_ResNet20_CIFAR-10'
l4_df = pd.read_pickle('results/L4/NUCLEO_L496ZG_01d_ResNet20_CIFAR-10_benchmarking_results_2020-07-20.pkl').sort_values(by=['model_type'])
f4_df = pd.read_pickle('results/F4/DISCO_F469NI_01d_ResNet20_CIFAR-10_benchmarking_results_2020-07-19.pkl').sort_values(by=['model_type'])
f7_df = pd.read_pickle('results/F7/NUCLEO_F767ZI_01d_ResNet20_CIFAR-10_benchmarking_results_2020-07-22.pkl').sort_values(by=['model_type'])

resnet_df = l4_df.append(f4_df)
resnet_df = resnet_df.append(f7_df)

Replace '_' with '-' as tikzplotlib does not convert them to correct LaTeX code.

In [4]:
lenet_df['model_type'] = lenet_df['model_type'].str.replace('_','-')
resnet_df['model_type'] = resnet_df['model_type'].str.replace('_','-')

lenet_df['MCU'] = lenet_df['MCU'].str.replace('_','-')
resnet_df['MCU'] = resnet_df['MCU'].str.replace('_','-')

In [5]:
resnet_df[resnet_df['MCU'] == 'NUCLEO-L496ZG']['tflu_mcu_benchmark_mean_50'].fillna(0)

11           0.00
17           0.00
16           0.00
22           0.00
10           0.00
23           0.00
4            0.00
5            0.00
14    25996287.54
21    55190048.88
8     37477172.26
9     37477033.18
3     12036431.88
20    55608173.42
15    25995365.02
2     12036423.38
19     2844980.80
18    26584332.72
6     12090503.92
13     2839520.76
12    26580787.34
7      2223333.86
1      2217008.96
0     12088462.60
Name: tflu_mcu_benchmark_mean_50, dtype: float64

Available columns

In [6]:
resnet_df.columns

Index(['time', 'MCU', 'model', 'mbed-dir', 'cmsis-nn', 'arena_size',
       'compiler_optimization', 'FPU_status', 'MACs', 'pruned', 'weights',
       'activations', 'model_type', 'model_size', 'model_size_reduction',
       'model_size_gzip', 'model_size_reduction_gzip', 'binary_size',
       'input_details_dtype', 'input_details_shape', 'output_details_dtype',
       'output_details_shape', 'tfl_interpreter_accuracy',
       'tfl_interpreter_loss_crossentropy', 'tfl_interpreter_loss_meansquared',
       'inferences_per_cycle', 'tflu_mcu_benchmark_single',
       'tflu_mcu_benchmark_mean', 'tflu_mcu_benchmark_std',
       'tflu_mcu_accuracy', 'tflu_mcu_loss_crossentropy',
       'tflu_mcu_loss_meansquared', 'tflu_mcu_accuracy_50',
       'tflu_mcu_benchmark_mean_50', 'tflu_mcu_benchmark_std_50', 'error'],
      dtype='object')

# Comparing MCUs for LeNet

### Fastest model (int8, cmsis-nn, Ofast, FPU)

In [7]:
hue = 'MCU'

filter_dict = {
    'compiler_optimization': '-Ofast',
    'cmsis-nn': 'cmsis-nn',
    'FPU_status': 1
}

for df in [lenet_df, resnet_df]:
        model_name = get_model_name(df['model'].unique()[0])


        filter = (df['weights'] == df['activations'])

        filter_descr = ''
        for key in filter_dict:
            filter = filter & (df[key] == filter_dict[key])
            filter_descr += (str(filter_dict[key]) + '_')



        df_filtered = df.loc[filter]
        


        # let's sort the filtered df so that the order in the plots match
        df_filtered = df_filtered.sort_values(by=['model_type', 'MCU'])

        try:
            df_filtered.loc[:,('tflu_mcu_benchmark_mean_50')] /= 1000
            fig = sns.catplot(y='model_type', x='tflu_mcu_benchmark_mean_50', kind='bar', hue=hue, data=df_filtered, height=4, aspect=16/9, hue_order=["NUCLEO-L496ZG", "DISCO-F469NI", "NUCLEO-F767ZI"])
            display(df_filtered[['model_type', 'tflu_mcu_benchmark_mean_50']].drop_duplicates())

        except KeyError:
            df_filtered.loc[:,('tflu_mcu_benchmark_mean')] /= 1000
            fig = sns.catplot(y='model_type', x='tflu_mcu_benchmark_mean', kind='bar', hue=hue, data=df_filtered, height=4, aspect=16/9, hue_order=["NUCLEO-L496ZG", "DISCO-F469NI", "NUCLEO-F767ZI"])
            display(df_filtered[['model_type','tflu_mcu_benchmark_mean']].drop_duplicates())


        #plt.title(model_name  + json.dumps(filter_dict, indent=4)[1:-1])

        plt.xlabel("Inference Latency [ms]")
        plt.ylabel("Model Type")
        fig.set_yticklabels(['\\texttt{float32}', '\\texttt{int8}'])

        #plt.legend()
        fig._legend.texts[0].set_text("L4")
        fig._legend.texts[1].set_text("F4")
        fig._legend.texts[2].set_text("F7")


        filename = f'figures/latency/perModel/{model_name}_hue-{hue}__{filter_descr}'

        print(filename)

        plt.tight_layout()
        plt.show()
        plt.savefig(filename + '.pdf')
        tikzplotlib.save(filename + '.tex', axis_width=AXIS_WIDTH, axis_height=AXIS_HEIGHT)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,model_type,tflu_mcu_benchmark_mean
13,W-float32-A-float32,59.5089206
1,W-float32-A-float32,19.9671834
1,W-float32-A-float32,113.1516721
15,W-int8-A-int8,16.1060121
3,W-int8-A-int8,8.0455337
3,W-int8-A-int8,36.3913420


figures/latency/perModel/LeNet-MNIST_hue-MCU__-Ofast_cmsis-nn_1_


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,model_type,tflu_mcu_benchmark_mean_50
5,W-float32-A-float32,6151.93348
3,W-float32-A-float32,1613.39756
5,W-float32-A-float32,NaN
1,W-int8-A-int8,984.31760
1,W-int8-A-int8,449.30214
1,W-int8-A-int8,2217.00896


figures/latency/perModel/ResNet20_CIFAR-10_hue-MCU__-Ofast_cmsis-nn_1_


## int8 acceleration per MCU

In [8]:
hue = 'model_type'

filter_dict = {
    'compiler_optimization': '-Ofast',
    'cmsis-nn': 'cmsis-nn',
    'FPU_status': 1
}

for df in [lenet_df, resnet_df]:
        model_name = get_model_name(df['model'].unique()[0])


        filter = (df['weights'] == df['activations'])

        filter_descr = ''
        for key in filter_dict:
            filter = filter & (df[key] == filter_dict[key])
            filter_descr += (str(filter_dict[key]) + '_')



        df_filtered = df.loc[filter]

        # let's sort the filtered df so that the order in the plots match
        df_filtered = df_filtered.sort_values(by=['model_type','cmsis-nn'])

        try:
            df_filtered.loc[:,('tflu_mcu_benchmark_mean_50')] /= 1000
            fig = sns.catplot(y='MCU', x='tflu_mcu_benchmark_mean_50', kind='bar', hue=hue, data=df_filtered, height=4, aspect=16/9, order=["NUCLEO-L496ZG", "DISCO-F469NI", "NUCLEO-F767ZI"])
            display(df_filtered[['MCU', 'tflu_mcu_benchmark_mean_50']].drop_duplicates())

        except KeyError:
            df_filtered.loc[:,('tflu_mcu_benchmark_mean')] /= 1000
            fig = sns.catplot(y='MCU', x='tflu_mcu_benchmark_mean', kind='bar', hue=hue, data=df_filtered, height=4, aspect=16/9, order=["NUCLEO-L496ZG", "DISCO-F469NI", "NUCLEO-F767ZI"])
            display(df_filtered[['MCU','tflu_mcu_benchmark_mean']].drop_duplicates())


        plt.title(model_name  + json.dumps(filter_dict, indent=4)[1:-1])

        plt.xlabel("Inference latency [ms]")
        plt.ylabel("Model type")

        fig.set_yticklabels(['L4', 'F4', 'F7'])

        fig._legend.texts[0].set_text("\\texttt{float32}")
        fig._legend.texts[1].set_text("\\texttt{int8}")
        
        filename = f'figures/latency/perModel/{model_name}_hue-{hue}__{filter_descr}'

        print(filename)

        plt.tight_layout()
        plt.show()
        plt.savefig(filename + '.pdf')
        tikzplotlib.save(filename + '.tex', axis_width=AXIS_WIDTH, axis_height=AXIS_HEIGHT)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,MCU,tflu_mcu_benchmark_mean
1,NUCLEO-L496ZG,113.1516721
13,DISCO-F469NI,59.5089206
1,NUCLEO-F767ZI,19.9671834
3,NUCLEO-L496ZG,36.3913420
15,DISCO-F469NI,16.1060121
3,NUCLEO-F767ZI,8.0455337


figures/latency/perModel/LeNet-MNIST_hue-model_type__-Ofast_cmsis-nn_1_


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,MCU,tflu_mcu_benchmark_mean_50
5,NUCLEO-L496ZG,NaN
5,DISCO-F469NI,6151.93348
3,NUCLEO-F767ZI,1613.39756
1,NUCLEO-L496ZG,2217.00896
1,DISCO-F469NI,984.31760
1,NUCLEO-F767ZI,449.30214


figures/latency/perModel/ResNet20_CIFAR-10_hue-model_type__-Ofast_cmsis-nn_1_


## cmsis-nn acceleration per MCU

In [9]:
hue = 'cmsis-nn'

filter_dict = {
    'compiler_optimization': '-Ofast',
    'weights': 'int8',
    'FPU_status': 1
}

for df in [lenet_df, resnet_df]:
        model_name = get_model_name(df['model'].unique()[0])


        filter = (df['weights'] == df['activations'])

        filter_descr = ''
        for key in filter_dict:
            filter = filter & (df[key] == filter_dict[key])
            filter_descr += (str(filter_dict[key]) + '_')



        df_filtered = df.loc[filter]

        # let's sort the filtered df so that the order in the plots match
        df_filtered = df_filtered.sort_values(by=['model_type','cmsis-nn'])

        try:
            df_filtered.loc[:,('tflu_mcu_benchmark_mean_50')] /= 1000
            fig = sns.catplot(y='MCU', x='tflu_mcu_benchmark_mean_50', kind='bar', hue=hue, data=df_filtered, height=4, aspect=16/9)
            display(df_filtered[['MCU', 'tflu_mcu_benchmark_mean_50']].drop_duplicates())

        except KeyError:
            df_filtered.loc[:,('tflu_mcu_benchmark_mean')] /= 1000
            fig = sns.catplot(y='MCU', x='tflu_mcu_benchmark_mean', kind='bar', hue=hue, data=df_filtered, height=4, aspect=16/9)
            display(df_filtered[['MCU','tflu_mcu_benchmark_mean']].drop_duplicates())


#        plt.title(model_name  + json.dumps(filter_dict, indent=4)[1:-1])

        plt.xlabel("Inference Latency [ms]")
        plt.ylabel("Model Type")

        fig.set_yticklabels(['L4', 'F4', 'F7'])

        fig._legend.texts[0].set_text("\\texttt{cmsis-nn}")
        fig._legend.texts[1].set_text("\\textit{none}")
        
        filename = f'figures/latency/perModel/{model_name}_hue-{hue}__{filter_descr}'

        print(filename)

        plt.tight_layout()
        plt.show()
        plt.savefig(filename + '.pdf')
        tikzplotlib.save(filename + '.tex', axis_width=AXIS_WIDTH, axis_height=AXIS_HEIGHT)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,MCU,tflu_mcu_benchmark_mean
3,NUCLEO-L496ZG,36.3913420
15,DISCO-F469NI,16.1060121
3,NUCLEO-F767ZI,8.0455337
2,NUCLEO-L496ZG,120.6695127
14,DISCO-F469NI,65.5978391
2,NUCLEO-F767ZI,17.8879866


figures/latency/perModel/LeNet-MNIST_hue-cmsis-nn__-Ofast_int8_1_


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,MCU,tflu_mcu_benchmark_mean_50
1,NUCLEO-L496ZG,2217.00896
1,DISCO-F469NI,984.31760
1,NUCLEO-F767ZI,449.30214
0,NUCLEO-L496ZG,12088.46260
0,DISCO-F469NI,6506.26076
0,NUCLEO-F767ZI,1119.74508


figures/latency/perModel/ResNet20_CIFAR-10_hue-cmsis-nn__-Ofast_int8_1_
